In [1]:
import zipfile, glob, os
from functools import reduce

import pandas as pd
import numpy as np

data_path = "data/eurostat"

In [2]:
def eurostat_reader(file_path, na_proportion = 0.8, 
                    cols_to_drop={"all": {'UNIT'},
                                  "arrivals_at_tourist_accommodation_establishments.zip" : {'C_RESID'},
                                  "average_length_of_stay_at_hospitals.zip": {'SEX', 'AGE', 'ICD10'},
                                  "crude_death_rate.zip": {'SEX', 'AGE'},
                                  "disposable_income_per_inhabitant.zip": {'DIRECT'},
                                  "percentage_education_attainment.zip": {'SEX', 'AGE'},
                                  "population_numbers.zip": {'SEX', 'AGE'}}):

    with zipfile.ZipFile(file_path, 'r') as zip_file:
        for file in zip_file.namelist():
            if 'Data' in file:
                with zip_file.open(file) as data_file:
                    df = pd.read_csv(data_file, encoding="ISO-8859-1")
    
    base_file_path = os.path.basename(file_path)        
    # Drop desired columns
    if base_file_path in cols_to_drop.keys():
        if (len(cols_to_drop[base_file_path].intersection(df.columns)) > 0):
            df = df.drop(columns=cols_to_drop[base_file_path].intersection(df.columns))
    df = df.drop(columns=cols_to_drop["all"])
        
    # Clean Value column and convert to float
    df["Value"] = df["Value"].apply(
        lambda x: x.replace(",", "")).replace({":": None}, regex=False).astype(np.float32)
    
    # Check if there is no data for certain years and then drop those years
    na_check = df.set_index('TIME')['Value'].isna().all(level=0)
    if len(na_check.index[na_check]) > 0:
        df = df.set_index('TIME').drop(list(na_check.index[na_check])).reset_index()
    
    # Find the column containing the relevant values to spread on
    value_col = [col for col in df.columns if col not in {"TIME", "GEO", "Value"}]
    
    if len(value_col) > 1:
        raise ValueError(f"Too many columns available to spread on for file '{file_path}', "
                         f"namely {value_col}. Check the data and add columns to remove to cols_to_drop.")  
    
    # Pivot from long to wide
    if len(value_col) == 1:
        df = df.pivot_table(index=['TIME','GEO'], columns=value_col[0], values='Value').reset_index()
        del df.columns.name
    
    # Drop fully NA rows and columns
    df = df.dropna(how='all').dropna(axis=1, how='all')
    na_props = df.isna().sum().divide(df.apply(len))
    df = df.drop(columns=na_props[na_props > na_proportion].index)
    
    if len(value_col) == 0:
        df = df.rename(columns={'Value': base_file_path.replace('.zip', '')})
    
    return df

In [3]:
dfs = []

for file in glob.glob(f"{data_path}/*.zip"):
    if 'by_rail_by_loading_unloading_region' in file:
        continue
    if 'TOCLEAN' in file:
        # TODO: Skip these files for now. These are per NUTS3 region and need to be aggregated.
        continue
    
    df = eurostat_reader(file)
    dfs.append(df)

In [4]:
# Merge (outer join) the list of DataFrames into one big DataFrame
df_merged = reduce(lambda x, y: pd.merge(x, y, on=['TIME', 'GEO'], how='outer'), dfs).sort_values(by=['TIME', 'GEO'])

# Drop extra regions
extra_region = (df_merged["GEO"] == "Extra-Regio NUTS 1") | (df_merged["GEO"] == "Extra-Regio NUTS 2")
df_merged = df_merged.drop(extra_region[extra_region].index)

# Propagate constant values over years for each region - Total area
df_merged = df_merged.set_index('GEO')
constant_columns = ['Total area']

for region in df_merged.index.unique():
    for col in constant_columns:
        value = df_merged.loc[region, col][df_merged.loc[region, col].notna()]

        try:
            df_merged.loc[region, col] = [value for i in range(len(df_merged.loc[region, col]))]
        except ValueError:
            print(f"Region: {region}, Value: {value}")
        
df_merged = df_merged.reset_index()

In [5]:
df_merged = df_merged.rename(columns={
    'TIME': 'time',
    'GEO': 'region',
    'Freight and mail loaded': 'air_freight_loaded',
    'Freight and mail unloaded': 'air_freight_unloaded',
    'Passengers carried (arrival)': 'air_passengers_arrived',
    'Passengers carried (departures)': 'air_passengers_departed',
    'Total area': 'area',
    'Hotels; holiday and other short-stay accommodation; camping grounds, recreational vehicle parks and trailer parks': 'tourist_arrivals',
    'In-patient average length of stay (in days)': 'hospital_stay',
    'All causes of death (A00-Y89) excluding S00-T98': 'death_rate_all',
    'Human immunodeficiency virus [HIV] disease': 'death_rate_hiv',
    'Influenza (including swine flu)': 'death_rate_influenza',
    'Tuberculosis': 'death_rate_tb',
    'Viral hepatitis and sequelae of viral hepatitis': 'death_rate_viralhepatitis',
    'Disposable income, net': 'disposable_income',
    'Medical doctors': 'medical_doctors',
    'Nurses and midwives': 'nurses_midwives',
    'Available beds in hospitals': 'available_beds',
    'Curative care beds in hospitals (HP.1)': 'curative_care_beds',
    'Long-term care beds in hospitals (HP.1)': 'longterm_care_beds',
    'Other beds in hospitals (HP.1)': 'other_beds',
    'Psychiatric care beds in hospitals (HP.1)': 'psychiatric_care_beds',
    'Rehabilitative care beds in hospitals (HP.1)': 'rehabilitative_care_beds',
    'Internet use: interaction with public authorities (last 12 months)': 'internet_contact_authorities',   
    'Electrified railway lines': 'length_electrified_railway',
    'Motorways': 'length_motorways',
    'Navigable canals': 'length_canals',
    'Navigable rivers': 'length_rivers',
    'Other roads': 'length_other_roads',
    'Railway lines with double and more tracks': 'length_large_railway',
    'Total railway lines': 'length_railway',
    'Freight loaded': 'maritime_freight_loaded',
    'Freight unloaded': 'maritime_freight_unloaded',
    'Passengers disembarked': 'maritime_passengers_disembarked',
    'Passengers embarked': 'maritime_passengers_embarked',
    'Median age of population': 'median_age',
    'Less than primary, primary and lower secondary education (levels 0-2)': 'lower_education',
    'Upper secondary and post-secondary non-tertiary education (levels 3 and 4)': 'higher_education',
    'Tertiary education (levels 5-8)': 'tertiary_education'
})

In [6]:
df_merged.head()

,region,time,air_freight_loaded,air_freight_unloaded,air_passengers_arrived,air_passengers_departed,area,tourist_arrivals,hospital_stay,broadband_access,...,maritime_freight_unloaded,maritime_passengers_disembarked,maritime_passengers_embarked,median_age,lower_education,tertiary_education,higher_education,population_density,population_numbers,risk_of_poverty_or_social_exclusion
0,Abruzzo,1990,NaN,NaN,NaN,NaN,10833.0,920164.0,NaN,NaN,...,NaN,NaN,NaN,37.400002,NaN,NaN,NaN,115.300003,1243014.0,NaN
1,Basilicata,1990,NaN,NaN,NaN,NaN,10073.0,187695.0,NaN,NaN,...,NaN,NaN,NaN,33.700001,NaN,NaN,NaN,61.099998,611029.0,NaN
2,Calabria,1990,NaN,NaN,NaN,NaN,15222.0,676446.0,NaN,NaN,...,NaN,NaN,NaN,32.099998,NaN,NaN,NaN,137.800003,2080660.0,NaN
3,Campania,1990,NaN,NaN,NaN,NaN,13670.0,2146643.0,NaN,NaN,...,NaN,NaN,NaN,30.500000,NaN,NaN,NaN,413.000000,5607965.0,NaN
4,Centro (IT),1990,NaN,NaN,NaN,NaN,58085.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_merged.to_csv(f"{data_path}/merged_eurostat.csv", index=False)

# Inter-city railroad connections

In [8]:
file_path = (f"{data_path}/passengers_by_rail_by_loading_unloading_region.zip")

with zipfile.ZipFile(file_path, 'r') as zip_file:
        for file in zip_file.namelist():
            if 'Data' in file:
                with zip_file.open(file) as data_file:
                    df = pd.read_csv(data_file, encoding="ISO-8859-1")

# GEO is constant, simply "Italy"
df = df.drop(columns=['UNIT', 'GEO'])

# Clean Value column and convert to float
df["Value"] = df["Value"].apply(
    lambda x: x.replace(",", "")).replace({":": None}, regex=False).astype(np.float32)

# Check if there is no data for certain years and then drop those years
na_check = df.set_index('TIME')['Value'].isna().all(level=0)
if len(na_check.index[na_check]) > 0:
    df = df.set_index('TIME').drop(list(na_check.index[na_check])).reset_index()

# Pivot from long to wide
df = df.pivot_table('Value', index=['TIME', 'C_LOAD'], columns='C_UNLOAD').reset_index()
del df.columns.name

df

,TIME,C_LOAD,Abruzzo,Basilicata,Calabria,Campania,Emilia-Romagna,Friuli-Venezia Giulia,Lazio,Liguria,...,Piemonte,Provincia Autonoma di Bolzano/Bozen,Provincia Autonoma di Trento,Puglia,Sardegna,Sicilia,Toscana,Umbria,Valle d'Aosta/Vallée d'Aoste,Veneto
0,2005,Abruzzo,4627401.0,794.0,3302.0,1060.0,NaN,NaN,NaN,1993.0,...,30669.0,NaN,NaN,124790.0,NaN,845.0,NaN,NaN,170.0,NaN
1,2005,Basilicata,658.0,1646354.0,3815.0,16370.0,NaN,NaN,NaN,948.0,...,5791.0,NaN,NaN,4922.0,NaN,1153.0,NaN,NaN,56.0,NaN
2,2005,Calabria,3671.0,5232.0,5656993.0,224269.0,NaN,NaN,NaN,57462.0,...,166490.0,NaN,NaN,49355.0,NaN,33851.0,NaN,NaN,750.0,NaN
3,2005,Campania,1403.0,13866.0,238684.0,21613624.0,NaN,NaN,NaN,99233.0,...,146922.0,NaN,NaN,101706.0,NaN,108708.0,NaN,NaN,357.0,NaN
4,2005,Liguria,2717.0,789.0,57734.0,96512.0,NaN,NaN,NaN,33906636.0,...,175947.0,NaN,NaN,15585.0,NaN,44230.0,NaN,NaN,49.0,NaN
5,2005,Lombardia,135084.0,15527.0,263387.0,516190.0,NaN,NaN,NaN,970218.0,...,774026.0,NaN,NaN,601418.0,8.0,167260.0,NaN,NaN,9.0,NaN
6,2005,Molise,11165.0,105.0,828.0,162.0,NaN,NaN,NaN,414.0,...,9640.0,NaN,NaN,19275.0,NaN,233.0,NaN,NaN,23.0,NaN
7,2005,Piemonte,30859.0,7109.0,169041.0,145978.0,NaN,NaN,NaN,187057.0,...,42741720.0,NaN,NaN,187003.0,19.0,88188.0,NaN,NaN,NaN,NaN
8,2005,Puglia,124902.0,6639.0,46225.0,99942.0,NaN,NaN,NaN,15804.0,...,181746.0,NaN,NaN,13656459.0,NaN,10924.0,NaN,NaN,859.0,NaN
9,2005,Sardegna,NaN,NaN,NaN,1.0,NaN,NaN,NaN,6.0,...,33.0,NaN,NaN,NaN,3991017.0,NaN,NaN,NaN,1.0,NaN


In [9]:
df.to_csv(f"{data_path}/interregion_railroad_travel.csv", index=False)